In [12]:
from pprint import pprint
from copy import deepcopy
from termcolor import colored
import math

In [13]:
labirinto_draw = ''
with open('Modelos\maze01.txt', 'r') as fp:
    labirinto_draw = fp.readlines()

In [14]:
labirinto = []
for linha in labirinto_draw:
    aux = []
    for coluna in linha:
        if coluna == ' ':
            aux.append(0)
        elif coluna == 'E':
            aux.append(1)
        elif coluna == 'S':
            aux.append(2)
        else:
            aux.append(9)   
    labirinto.append(aux) 

In [22]:
class Estado:
    def __init__(self, posicao=[], custo = 0, acao = None, pai = None):
        self.posicao = posicao
        self.custo = custo
        self.acao = acao
        self.pai = pai
    def get_custo(self):
        return self.custo
    def get_fn(self):
        return self.fn

In [23]:
def Dist_Euclidiana(x1,y1,x2,y2):
    return math.sqrt(math.pow((x2-x1),2)+math.pow((y2-y1),2))

In [24]:
def Movimentos_BG(posicao_atual,saida,labirinto):
    direita=None
    esquerda=None
    cima=None
    baixo=None
    lista=[]
    
    #Direita
    if(labirinto[posicao_atual.posicao[0]][posicao_atual.posicao[1]+1]==0):
        posicao=[posicao_atual.posicao[0],posicao_atual.posicao[1]+1]
        custo=Dist_Euclidiana(posicao[0],posicao[1],saida[0],saida[1])
        acao="→"
        pai=posicao_atual
        direita=Estado(posicao,custo,acao,pai)
    #Esquerda
    if(posicao_atual.posicao[1]!=0):
        if(labirinto[posicao_atual.posicao[0]][posicao_atual.posicao[1]-1]==0):
            posicao=[posicao_atual.posicao[0],posicao_atual.posicao[1]-1]
            custo=Dist_Euclidiana(posicao[0],posicao[1],saida[0],saida[1])
            acao="←"
            pai=posicao_atual
            esquerda=Estado(posicao,custo,acao,pai)
    #Cima
    if(labirinto[posicao_atual.posicao[0]-1][posicao_atual.posicao[1]]==0):
        posicao=[posicao_atual.posicao[0]-1,posicao_atual.posicao[1]]
        custo=Dist_Euclidiana(posicao[0],posicao[1],saida[0],saida[1])
        acao="↑"
        pai=posicao_atual
        cima=Estado(posicao,custo,acao,pai)
    #Baixo
    if(labirinto[posicao_atual.posicao[0]+1][posicao_atual.posicao[1]]==0):
        posicao=[posicao_atual.posicao[0]+1,posicao_atual.posicao[1]]
        custo=Dist_Euclidiana(posicao[0],posicao[1],saida[0],saida[1])
        acao="↓"
        pai=posicao_atual
        baixo=Estado(posicao,custo,acao,pai)
    #Verificações
    if(direita!=None):
        lista.append(direita)
    if(esquerda!=None):
        lista.append(esquerda)
    if(cima!=None):
        lista.append(cima)
    if(baixo!=None):
        lista.append(baixo)
    lista=sorted(lista,key=Estado.get_custo,reverse=True)    
    return [lista.pop()]

In [25]:
def imprimir_solucao(estado_atual,historico):
    arvore=[]
    arvore.append(estado_atual)
    historico=sorted(historico,key=Estado.get_custo)
    while(estado_atual.pai!=None):
        for i in range(len(historico)):
            if(historico[i].posicao==estado_atual.pai.posicao or estado_atual.pai.posicao==None):
                arvore.insert(0,historico[i])
                estado_atual=historico[i]
                break
    return arvore

In [26]:
def imprimir_caminho(posicao_atual,historico,labirinto_draw):
    caminho=False
    historico=imprimir_solucao(posicao_atual,historico)
    for linha in range(len(labirinto_draw)):
        for coluna in range(len(labirinto_draw[0])):
            if(coluna==len(labirinto_draw[0])):
                print(labirinto_draw[linha][coluna])
            else:
                for i in range(len(historico)):
                    if([linha,coluna]==historico[i].posicao):
                        caminho=True
                        if(historico[i].acao==None):
                            print(colored("E", 'red'),end="")
                        elif(historico[i].custo==0):
                            print(colored("S", 'red'),end="")
                        else:
                            print(colored(historico[i].acao, 'red'),end="")
                        break
                if(caminho==False):
                    print(labirinto_draw[linha][coluna],end="")
                else:
                    caminho=False               
    
    

In [27]:
def busca_gulosa(labirinto,labirinto_draw):
    entrada=[]
    saida=[]
    for linha in range(len(labirinto)):
        for coluna in range(len(labirinto[0])):
            if(labirinto[linha][coluna]==1):
                entrada.append(linha)
                entrada.append(coluna)
            if(labirinto[linha][coluna]==2):
                saida.append(linha)
                saida.append(coluna)
    fronteira=[]
    historico=[]
    novo_movimento=True
    solucao_encontrada=False
    solucao=Estado(saida,0)
    posicao_atual=Estado(entrada,Dist_Euclidiana(entrada[0],entrada[1],saida[0],saida[1]))
    historico.append(posicao_atual)#deletar
    while(solucao_encontrada != True):        
        movimentos=Movimentos_BG(posicao_atual,saida,labirinto)
        for i in range(len(movimentos)):
            for j in range(len(historico)):
                if(movimentos[i].posicao==historico[j].posicao):
                    novo_movimento=False
                    break
            if(novo_movimento==True):
                if(movimentos[i].posicao==solucao.posicao):
                    solucao_encontrada=True
                    conjunto_solucao=deepcopy(movimentos[i])
                    break
                fronteira.append(movimentos[i])
                historico.append(movimentos[i])
            novo_movimento=True
        if(solucao_encontrada!=True):
            if(fronteira != []):
                fronteira=sorted(fronteira,key=Estado.get_custo,reverse=True)
                posicao_atual=fronteira.pop()
            else:                
                imprimir_caminho(posicao_atual,historico,labirinto_draw)
                print(colored("Sem mais caminhos possiveis!",'red'))
                break
        else:
            imprimir_caminho(posicao_atual,historico,labirinto_draw)
            print(colored("Solução Encontrada",'green'))
            

In [28]:
busca_gulosa(labirinto,labirinto_draw)

+--+--+--+--+--+--+--+--+--+--+
E→→→→→|           |     |     |
+  + ↓+  +--+--+  +--+  +  +  +
|  | ↓|  |     |  |        |  |
+  + ↓+--+ ↑+  +  +  +  +--+--+
|  | ↓→→→→→→|        |  |     |
+  +--+  +--+--+--+--+  +  +  +
|     |  |        |     |  |  |
+--+  +  +  +--+  +  +--+  +  +
|  |  |     |     |     |  |  |
+  +  +--+--+  +--+--+  +--+  +
|     |        |     |        |
+--+--+  +--+--+  +  +--+  +--+
|     |  |     |  |     |     |
+--+  +  +--+  +  +--+  +--+--+
|     |  |        |  |        |
+  +--+  +  +--+--+  +--+--+  +
|        |  |        |        |
+  +  +--+  +  +--+--+  +--+--+
|  |        |                 S
+--+--+--+--+--+--+--+--+--+--+
Sem mais caminhos possiveis!
